In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import ultralytics

In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="iqWudABIjfXqATDPAmLo")
project = rf.workspace("custome-yolo").project("all-weapons-data-jqimy")
version = project.version(1)
dataset = version.download("yolov8",location="C:/Users/PMLS/Downloads")


loading Roboflow workspace...
loading Roboflow project...


: 

In [3]:
train_images_path="D:/ML/Tasks/CNN/All-weapons-data-1/train/images"
train_labels_path="D:/ML/Tasks/CNN/All-weapons-data-1/train/labels"
test_images_path="D:/ML/Tasks/CNN/All-weapons-data-1/test/images"
test_labels_path="D:/ML/Tasks/CNN/All-weapons-data-1/test/labels"
validation_images_path="D:/ML/Tasks/CNN/All-weapons-data-1/valid/images"
validation_labels_path="D:/ML/Tasks/CNN/All-weapons-data-1/valid/labels"

We will first check if all the train , test and valid set have correct corresponding images and labels or not


In [4]:

def check_image_label_pairs(images_path, labels_path):
    image_names = {os.path.splitext(f)[0] for f in os.listdir(images_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))}
    label_names = {os.path.splitext(f)[0] for f in os.listdir(labels_path) if f.lower().endswith('.txt')}

    missing_labels = image_names - label_names
    missing_images = label_names - image_names

    print(f"Total images: {len(image_names)}")
    print(f"Total labels: {len(label_names)}")
    print("")

    if missing_labels:
        print(f"{len(missing_labels)} images have no label file:")
        for name in sorted(list(missing_labels))[:10]:
            print("  -", name)
        if len(missing_labels) > 10:
            print(f"  ... and {len(missing_labels)-10} more.")
    else:
        print("All images have matching labels.")

    print("")

    if missing_images:
        print(f"{len(missing_images)} labels have no image file:")
        for name in sorted(list(missing_images))[:10]:
            print("  -", name)
        if len(missing_images) > 10:
            print(f"  ... and {len(missing_images)-10} more.")
    else:
        print("All labels have matching images.")


In [5]:
check_image_label_pairs(train_images_path, train_labels_path)

Total images: 7396
Total labels: 7396

All images have matching labels.

All labels have matching images.


In [6]:
check_image_label_pairs(test_images_path, test_labels_path)

Total images: 593
Total labels: 593

All images have matching labels.

All labels have matching images.


In [7]:
check_image_label_pairs(validation_images_path, validation_labels_path)

Total images: 591
Total labels: 591

All images have matching labels.

All labels have matching images.


We dont have to worry about dimensions , yolo will take care of it automatically.

In [ ]:
import os
from collections import Counter

label_dir = "D:/ML/Tasks/CNN/All-weapons-data-1/train/labels"  # adjust path if needed
all_labels = []

for file in os.listdir(label_dir):
    if file.endswith(".txt"):
        with open(os.path.join(label_dir, file)) as f:
            for line in f:
                class_id = int(line.split()[0])
                all_labels.append(class_id)




<generator object <genexpr> at 0x000001A7E037D630>


In [15]:
print(Counter(all_labels))

Counter({2: 2588, 3: 2095, 5: 1263, 1: 820, 4: 699, 0: 663, 6: 88})


In [20]:
import os
import shutil

# Paths
paths = {
    "train": {"images": "D:/ML/Tasks/CNN/All-weapons-data-1/train/images",
              "labels": "D:/ML/Tasks/CNN/All-weapons-data-1/train/labels"},
    "test": {"images": "D:/ML/Tasks/CNN/All-weapons-data-1/test/images",
             "labels": "D:/ML/Tasks/CNN/All-weapons-data-1/test/labels"},
    "valid": {"images": "D:/ML/Tasks/CNN/All-weapons-data-1/valid/images",
              "labels": "D:/ML/Tasks/CNN/All-weapons-data-1/valid/labels"}
}

def rename_files(image_dir, label_dir, prefix, dry_run=False):
    images = sorted([f for f in os.listdir(image_dir) 
                     if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    
    rename_pairs = []
    for idx, img_name in enumerate(images, 1):
        new_name = f"{prefix}_{idx:04d}"
        img_ext = os.path.splitext(img_name)[1].lower()  # Normalize extension
        
        old_img_path = os.path.join(image_dir, img_name)
        new_img_path = os.path.join(image_dir, new_name + img_ext)
        
        # Check for corresponding label
        label_name = os.path.splitext(img_name)[0] + ".txt"
        old_label_path = os.path.join(label_dir, label_name)
        
        if os.path.exists(old_label_path):
            new_label_path = os.path.join(label_dir, new_name + ".txt")
            rename_pairs.append((old_img_path, new_img_path, old_label_path, new_label_path))
        else:
            print(f"⚠️  Warning: Label not found for {img_name}")
            rename_pairs.append((old_img_path, new_img_path, None, None))
    
    if dry_run:
        print(f"\n📋 Preview for {prefix}:")
        for old_img, new_img, old_lbl, new_lbl in rename_pairs[:5]:  # Show first 5
            print(f"  {os.path.basename(old_img)} → {os.path.basename(new_img)}")
        print(f"  ... and {len(rename_pairs) - 5} more files")
        return
    
    # Actual renaming
    for old_img, new_img, old_lbl, new_lbl in rename_pairs:
        os.rename(old_img, new_img)
        if old_lbl and new_lbl:
            os.rename(old_lbl, new_lbl)
    
    print(f"✅ Renamed {len(rename_pairs)} files in {prefix} set")

# DRY RUN FIRST (preview changes)
# print("=== DRY RUN MODE ===")
# for set_name, dirs in paths.items():
#     rename_files(dirs["images"], dirs["labels"], prefix=set_name, dry_run=True)

# Uncomment below to actually rename:
print("\n=== ACTUAL RENAMING ===")
for set_name, dirs in paths.items():
    rename_files(dirs["images"], dirs["labels"], prefix=set_name, dry_run=False)
print("✅ Done!")


=== ACTUAL RENAMING ===
✅ Renamed 7396 files in train set
✅ Renamed 593 files in test set
✅ Renamed 591 files in valid set
✅ Done!


In [16]:
from ultralytics import YOLO

In [19]:

model = YOLO("yolov8n.yaml")  # just the architecture, no weights
model.train(
    data="./All-weapons-data-1/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    lr0=0.001,
    lrf=0.01,
    optimizer="SGD",
    momentum=0.937,
    weight_decay=0.0005,
    workers=6,
    name="weapons_train",
    project="./Yolo_results",
    augment=True,
    mosaic=True,
    mixup=True,
    flipud=0.5,
    fliplr=0.5,
    degrees=15,
    translate=0.1,
    scale=0.1
)



Ultralytics 8.3.226  Python-3.11.3 torch-2.9.0+cpu CPU (12th Gen Intel Core i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./All-weapons-data-1/data.yaml, degrees=15, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=True, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=True, multi_scale=False, name=weapons_train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=100, perspective=0.0, pl

KeyboardInterrupt: 